In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pylab as plt
from matplotlib import gridspec
import csv
import numpy as np
import pandas as pd
import os
import torch
from torchvision import transforms
from torchvision import datasets
import seaborn as sn
sn.set_context('poster')

In [ ]:
dats = pd.read_csv(os.path.join('perf', 'smnist_deepsith_11.csv'))
dats.columns = ['loss', 'test_perf', 'epoch', 'presnum', 'perf']
maxpres = 60000
dats['presnum_epoch'] = ((dats.presnum*64) + maxpres*dats.epoch)/maxpres
test_dats = pd.read_csv("perf/smnist_deepsith_test_11.csv")
test_dats['epoch'] = np.arange(test_dats.shape[0]) + 1
test_dats['test'] = test_dats['0']

In [ ]:
dato = pd.read_csv(os.path.join('perf', 'pmnist_deepsith_78.csv'))
dato.columns = ['loss', 'epoch', 'presnum', 'perf']
maxpres = 60000
dato['presnum_epoch'] = ((dato.presnum*64) + maxpres*dato.epoch)/maxpres
test_dato = pd.read_csv("perf/pmnist_deesith_test_78.csv")
test_dato.epoch = test_dato.epoch+1
test_dato.head()

In [ ]:
norm = transforms.Normalize((.1307,), (.3081,), )
batch_size = 400
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((.1307,), (.3081,))
                               ])
ds1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
train_loader=torch.utils.data.DataLoader(ds1,batch_size=batch_size, 
                                         num_workers=1, pin_memory=True, shuffle=True)


In [ ]:
# Same seed and supposed Permutation as the coRNN paper
torch.manual_seed(12008)
permute = torch.randperm(784)

In [ ]:
dat = next(enumerate(train_loader))[1]
dat[0].shape, dat[1].shape

In [ ]:
fig_dat = []
for i in range(10):
    fig_dat.append(dat[0][dat[1] == i][:10])
fig_dat = torch.cat(fig_dat, dim=0)
fig_dat = fig_dat.view(100,-1)#[:, permute]


In [ ]:
linew = 4
with sn.plotting_context("notebook", font_scale=2.8):
    fig2 = plt.figure(figsize=(20,18), )
    spec2 = gridspec.GridSpec(nrows=4, ncols=2, wspace=0.05, figure=fig2)


    ax = fig2.add_subplot(spec2[-2, 1])
    #fig, ax= plt.subplots(2,2,sharex='col', figsize=(12,10), sharey='row', )
    sn.lineplot(data=dato, x=dato.presnum_epoch, y='perf', ax=ax, linewidth=linew,
                color='darkblue', )
    sn.lineplot(data=test_dato, x='epoch', y='test', ax=ax, linewidth=linew,
                )
    ax.grid(True)
    ax.legend(["Training", "Test", 
               ],loc='lower right')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('')
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    plt.setp(ax.get_xticklabels(), visible=False)
    ax.set_xlim(0,90)#)
    ax.set_ylim(.99, 1.0005)

    ax = fig2.add_subplot(spec2[-1, 1], sharex=ax)
    sn.lineplot(data=dato, x=dato.presnum_epoch, y='loss', ax=ax,  linewidth=linew,
                color='darkblue', legend=False)
    ax.set_ylabel('Loss')

    ax.set_xlabel('Epoch')
    ax.yaxis.tick_right()
    
    ax.yaxis.set_label_position("right")
    ax.set_ylim(0,.01)#)
    ax.grid(True)


    
    ax = fig2.add_subplot(spec2[-2, 0])
    sn.lineplot(data=dats, x=dats.presnum_epoch, y='perf', ax=ax, linewidth=linew,
                color='darkblue', )
    sn.lineplot(data=test_dats, x='epoch', y='test', ax=ax, linewidth=linew,
                )
    ax.legend(["Training", "Test", 
               ],loc='lower right')
    plt.setp(ax.get_xticklabels(), visible=False)
    ax.grid(True)
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('')
    ax.set_xlim(0,60)#)
    ax.set_ylim(.99, 1.0005)

    ax = fig2.add_subplot(spec2[-1, 0], sharex=ax)
    sn.lineplot(data=dats, x=dats.presnum_epoch, y='loss', ax=ax,  linewidth=linew,
                color='darkblue', legend=False)
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylim(0,.01)#)
    ax.grid(True)


    
    ax = fig2.add_subplot(spec2[:-2, 0])
    ax.imshow(fig_dat.detach().cpu().numpy(), aspect='auto')
    ax.tick_params(axis=u'both', which=u'both',length=0)
    ax.set_yticks(np.arange(10,100,10))
    ax.set_xticks([])
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.grid(True)
    ax.set_title('sMNIST')

    ax = fig2.add_subplot(spec2[:-2, 1])
    ax.imshow(fig_dat[:, permute].detach().cpu().numpy(), aspect='auto')
    ax.tick_params(axis=u'both', which=u'both',length=0)
    ax.set_yticks(np.arange(10,100,10))
    ax.set_xticks([])
    ax.yaxis.tick_right()
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.grid(True)
    ax.set_title('psMNIST')

    plt.savefig('MNIST.pdf',
                bbox='tight',
                edgecolor=fig2.get_edgecolor(),
                facecolor=fig2.get_facecolor(),
                dpi=150
                )
    plt.savefig('MNIST.svg',
                bbox='tight',
                edgecolor=fig2.get_edgecolor(),
                facecolor=fig2.get_facecolor(),
                dpi=150
                )
    



